In [335]:
def print_vector(vec):
    for x in vec:
        print(x, end=" ")
    print()  # final newline


In [ ]:
def K(b, w, t): # Is 1 if you learn word w reading book b at time t
    # return [f"K({b},{w},{t})"]
    #delete later?# global v
    v_prime = []
    v_prime.append("(")

    # Now s_d goes from 0 to len(B[b]) - 1
    for s_d in range(len(B[b])):

        if w in B[b][s_d]:
            if len(B[b][s_d]) == 1: 
                # w is alone in s_d, so we learned w in this book
                v_prime.clear()
                #delete later?# v += [1]
                # So, we can return a 1, cause we will learn w in this book b
                return [1]

            v_prime.append("(")

            for omega in B[b][s_d]:
                if omega == w:
                    continue

                # k(b, s_d, omega, t)
                v_prime.append(f"k({b},{s_d},{omega},{t})")
                v_prime.append("AND")

            v_prime.append(")")
            v_prime.append("OR")

    v_prime.append(")")

    if v_prime == ["(",")"]:
        # we never found w in book b
        v_prime.clear()
        # So, we can return a 0, cause we won't learn w in this book b
        return [0]  
    
    #delete later?# v.extend(v_prime)
    
    return v_prime


In [337]:
# def l(w, t):
#     v.append("(")
#     if t == 0:
#         for b in range(len(B)):
#             if w in B[b][0]:
#                 k(b, 0, w, t)
#                 v.append("OR")

#     else:


#     v.append(")")

In [338]:
def k(b, s_d, omega, t):
    v.append(f"k({b},{s_d},{omega},{t})")
    #v.append(f"\"know word {omega} at b_{b}[{s_d}] at t={t}\"")
    return

In [339]:
def clean_constants(v):
    i = 0
    while i < len(v) - 2:
        a, op, b = v[i], v[i+1], v[i+2]

        # Case: 0 OR X → X
        if a == 0 and op == "OR":
            del v[i:i+2]   # remove 0, "OR"
            continue

        # Case: X OR 0 → X
        if op == "OR" and b == 0:
            del v[i+1:i+3]  # remove "OR", 0
            continue

        # Case: 1 AND X → X
        if a == 1 and op == "AND":
            del v[i:i+2]   # remove 1, "AND"
            continue

        # Case: X AND 1 → X
        if op == "AND" and b == 1:
            del v[i+1:i+3]  # remove "AND", 1
            continue

        i += 1


def clean_vector(v):
    i = 0
    while i < len(v) - 1:
        # If we find AND or OR followed by ")"
        if v[i] in ("AND", "OR") and v[i+1] == ")":
            del v[i]   # remove the AND/OR
            # do NOT advance i, because we want to re-check the same position
        else:
            i += 1
    clean_constants(v)


In [340]:
#  Creating the Modelling

v = [] # vector storing the ending expression

B = [ 
      [ [0] ] ,                          # b0
      [ [0,1] ] ,                        # b1
      [ [2,3], [4,0,2,3], [4] ] ,                   # b2
      [ [0,3], [2], [4,3,5], [4,0] ] ,   # b3
    ]

n_of_unknown_words = 6 #max number in B - 1

n_of_books_to_read = 2 # At most the amount of books; max value t+1 can have.

n_of_books = len(B)
# L = 

In [ ]:
# l creation / full_build()
# def full_build():
#     for b in range(len(B)):
#         for s_d in range(len(B[b])):
#             for w in B[b][s_d]:
#                 for t in range(n_of_books_to_read):

#                     L[][][]    
                    
                
                


# L = [w][t][b] of 
#   n° of words BY
#   n° of books to read (can be less than amount of books)) BY
#   n° of books

# L = [[[[] for b in range(len(B))] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]
l = [[[] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]

# inside L[w][t][b] there will be K(b,w,t)
# in each l[w][t] we will do OR between all b

for w in range(n_of_unknown_words):
    for t in range(n_of_books_to_read):
        l[w][t].append("(")

        if t != 0:
            l[w][t].extend(["(", f"l[{w}][{t-1}]", ")", "OR"])
        
        for b in range(n_of_books):
            big_k = K(b, w, t)
            if big_k == [0]: # If w wasn't in book b, we can skip book b
                continue

            l[w][t].extend(["(", "(", f"x[{b}]", "==", t, ")", "AND"])
            l[w][t] += big_k
            l[w][t].extend([")","OR"])
        
        l[w][t].append(")")
        clean_vector(l[w][t])

In [342]:
# v.clear()
# # K(b, w, t)
# K(2, 1, 1)
# clean_vector(v)

In [ ]:
print_vector(l[2][0])

( ( ( x[2] == 0 ) AND ( ( k(2,0,3,0) ) OR ( k(2,1,4,0) AND k(2,1,0,0) AND k(2,1,3,0) ) ) ) OR ( ( x[3] == 0 ) ) ) 
